In [ ]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic



# For webcam input:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open video device")

with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
    
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

  
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)
    print(results)
    break

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

In [ ]:
results.count

#Converting Mediapipe to Dataframe

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

face = pd.DataFrame()
pose = pd.DataFrame()
left_hand = pd.DataFrame()
right_hand = pd.DataFrame()

if results.face_landmarks is not None:
    for i, point in enumerate(results.face_landmarks.landmark):
        face.loc[i, ['x','y','z']] = [point.x, point.y,point.z]
if results.pose_landmarks is not None:
    for i, point in enumerate(results.pose_landmarks.landmark):
        pose.loc[i, ['x','y','z']] = [point.x, point.y,point.z]
if results.left_hand_landmarks is not None:
    for i, point in enumerate(results.left_hand_landmarks.landmark):
        left_hand.loc[i, ['x','y','z']] = [point.x, point.y,point.z]
if results.right_hand_landmarks is not None:
    for i, point in enumerate(results.right_hand_landmarks.landmark):
        right_hand.loc[i, ['x','y','z']] = [point.x, point.y,point.z]

#Create the Kaggle Type Format from our detected Landmarks

In [ ]:
face.plot(x='x', y='y',kind='scatter')


In [ ]:
landmarks = pd.DataFrame()

Indices in face

In [ ]:
face

In [ ]:
face.shape

In [ ]:
def create_frame_landmark_df(results,frame):
    #for having values and rows for every landmark index
    xyz_skel= xyz[['type','landmark_index']].drop_duplicates().reset_index(drop=True).copy()
    face = pd.DataFrame()
    pose = pd.DataFrame()
    left_hand = pd.DataFrame()
    right_hand = pd.DataFrame()

    if results.face_landmarks is not None:
        for i, point in enumerate(results.face_landmarks.landmark):
            face.loc[i, ['x','y','z']] = [point.x, point.y,point.z]
    if results.pose_landmarks is not None:
        for i, point in enumerate(results.pose_landmarks.landmark):
            pose.loc[i, ['x','y','z']] = [point.x, point.y,point.z]
    if results.left_hand_landmarks is not None:
        for i, point in enumerate(results.left_hand_landmarks.landmark):
            left_hand.loc[i, ['x','y','z']] = [point.x, point.y,point.z]
    if results.right_hand_landmarks is not None:
        for i, point in enumerate(results.right_hand_landmarks.landmark):
            right_hand.loc[i, ['x','y','z']] = [point.x, point.y,point.z]

    face = face.reset_index().rename(columns={'index':'landmark_index'}).assign(type='face')

    pose = pose.reset_index().rename(columns={'index':'landmark_index'}).assign(type='pose')

    left_hand = left_hand.reset_index().rename(columns={'index':'landmark_index'}).assign(type='left_hand')

    right_hand = right_hand.reset_index().rename(columns={'index':'landmark_index'}).assign(type='right_hand')

    landmarks = pd.concat([face,pose,left_hand,right_hand]).reset_index(drop=True)
    #So that skel will have landmarks even if they do not exist
    landmarks = xyz_skel.merge(landmarks, on =['type','landmark_index'], how='left')
    #to have actual unique frames
    landmarks = landmarks.assign(frame=frame)
    return landmarks

In [ ]:
landmarks = create_frame_landmark_df(results,0)

In [ ]:
pose.shape

In [ ]:
left_hand.shape

In [ ]:
right_hand.shape

#Sign Counts

In [ ]:
train['sign'].value_counts()

#Kaggle Competition Data

In [ ]:
train = pd.read_csv('/Users/aayus/MLProjects/ASL/asl-signs/train.csv')
pq_file = '/Users/aayus/MLProjects/ASL/asl-signs/train_landmark_files/16069/10042041.parquet'
xyz = pd.read_parquet(pq_file)

#Maximum Amount of Points to predict

In [ ]:
xyz.query('frame == 83')['type'].value_counts()

In [ ]:
train.query('sequence_id==10042041')

In [ ]:
ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)


In [ ]:
xyz_np = load_relevant_data_subset(pq_file)